In [1]:
import numpy as np
import pandas as pd

import string
import spacy 
import random 

import seaborn as sns
import matplotlib.pyplot as plt

import re

from spacy.lang.pt.stop_words import STOP_WORDS


In [2]:
base_train = pd.read_csv('twitter_base/Train50.csv', delimiter=';')
base_train.drop(['id', 'tweet_date', 'query_used'], axis=1, inplace=True)
base_train

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1
...,...,...
49995,:( é tão lindo que dói https://t.co/GqnpgyWWxB,0
49996,"@veraluciarj Pois é.. tenho problema c/ ""coisa...",0
49997,eu te amo tanto minja vidinha meu bem mais pre...,0
49998,@itsLary @jessboluda Pfvor :(,0


In [3]:
base_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  50000 non-null  object
 1   sentiment   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [4]:
base_test = pd.read_csv('twitter_base/Test.csv', delimiter=';')
base_test.drop(['id', 'tweet_date', 'query_used'], axis=1, inplace=True)
base_test

,tweet_text,sentiment
0,@Gazo1a Nossa! Muito obrigada :),1
1,@BerzGamer vai pa puta que te pariu :),1
2,QUER MAIS DESCONTOS? (14/08) ⭐⭐⭐⭐⭐ 🌐 Confira n...,1
3,"EU VOU PEGAR VCS, ME AJUDEM GALERA, PELO AMOR ...",1
4,Estávamos em casa do Zé e eu estava a morrer d...,1
...,...,...
4995,@ol_cdanilo parece livro de autoajuda :(,0
4996,@tatazoquita aaaaa sinto muito :((,0
4997,To começando a sentir dor de novo e meu irmão ...,0
4998,@ichbintw parece que no dia que toma a vacina ...,0


In [5]:
base_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  5000 non-null   object
 1   sentiment   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.3+ KB


50 000 registros para treino e 5000 para teste

In [6]:
pln = spacy.load('pt_core_news_sm')

In [7]:
def pre_processamento(texto):

    texto = texto.lower()

    # Nome do usuário
    texto = re.sub(r"@[A-Za-z0-9$-_@.&_+]+", " ", texto)

    # URLs - remover
    texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)
    
    # Emoticons -- :) , :(, :d
    
    lista_emocoes = {':)': 'emocaopositiva', ':d': 'emocaopositiva', ':(': 'emocaonegativa'}

    for emocao in lista_emocoes:
        texto = texto.replace(emocao, lista_emocoes[emocao])

    ### Lematizacao
    documento = pln(texto)
    lista = []
    for token in documento:
        lista.append(token.lemma_)

    ## Remoção de STOP WORDS e pontuações
    lista = [palavra for palavra in lista if palavra not in STOP_WORDS and palavra not in string.punctuation]
    lista = ' '.join(
        [ str(elemento) for elemento in lista if not elemento.isdigit() ]
        )

    return lista

#### Limpeza dos textos

In [8]:
base_train['tweet_text'] = base_train['tweet_text'].apply(pre_processamento)
base_test['tweet_text'] = base_test['tweet_text'].apply(pre_processamento)

In [9]:
base_train

,tweet_text,sentiment
0,pessoa certo vale azevedo emocaopositivo,1
1,precisamente contrário emocaopositiva vir c...,1
2,video hoje ... pensar falar meta de o csgo pag...,1
3,aaaaaaaar amei polaroids expressar to apaixona...,1
4,valorizar coração de o menininho vc diferente ...,1
...,...,...
49995,emocaonegativo lir dói,0
49996,.. problema c/ .. de o toc emocaonegativo e...,0
49997,amo minja vidinho precioso desculpa péssima ut...,0
49998,pfvor emocaonegativo,0


In [10]:
base_test

,tweet_text,sentiment
0,emocaopositivo,1
1,pa puta parir emocaopositiva,1
2,querer desconto 14/08 ⭐ ⭐ ⭐ ⭐ ⭐ 🌐 confira em o...,1
3,vcs ajudar galera por o amor butera emocaoposi...,1
4,casa de o zé morrer sono chegar casa fico sono...,1
...,...,...
4995,parecer livro autoajuda emocaonegativo,0
4996,aaaaar sinto emocaonegativo,0
4997,to começar sentir dor irmão sumir remédio emoc...,0
4998,parecer em o dia tomar vacina n beber emoca...,0


In [11]:
db_final =  []

for texto, emocao in zip(base_train['tweet_text'], base_train['sentiment']):

    if emocao == 1:
        dic = ({'ALEGRIA': True, 'MEDO': False})
    else:
        dic = ({'ALEGRIA': False, 'MEDO': True})

    db_final.append([texto, dic.copy()])
db_final[:10]


[['   pessoa certo vale azevedo emocaopositivo',
  {'ALEGRIA': True, 'MEDO': False}],
 ['   precisamente contrário emocaopositiva vir chuva vir disposição emocaopositivo',
  {'ALEGRIA': True, 'MEDO': False}],
 ['video hoje ... pensar falar meta de o csgo pagina    alguir sugestao querer falar de o cenario nacional emocaopositivo',
  {'ALEGRIA': True, 'MEDO': False}],
 ['aaaaaaaar amei polaroids expressar to apaixonar vdd ✨ 💖 🎈 🎉 🎊 esperar outro pessoa conseguir ganhar emocaopositivar  ',
  {'ALEGRIA': True, 'MEDO': False}],
 ['valorizar coração de o menininho vc diferente sorrir de isso ❤ ️ — valorizar mundo em o vida diferente emocaopositivo  ',
  {'ALEGRIA': True, 'MEDO': False}],
 ['   amiga to emocaopositivar', {'ALEGRIA': True, 'MEDO': False}],
 ['   bravo jairo emocaopositivo', {'ALEGRIA': True, 'MEDO': False}],
 ['     socorro ksoaksosks achar to em o caixinha som :p',
  {'ALEGRIA': True, 'MEDO': False}],
 ['   gostar fazenda emocaopositivo morro medo galinha kkkkkkk',
  {'ALEGR

## Classificador - Não funcionando.

In [12]:
# NN
modelo = spacy.blank('pt') #  ... = 'pt' 

# Labels 
categorias = modelo.add_pipe('textcat') # text category
categorias.add_label("POSITIVO")
categorias.add_label("NEGATIVO")

# modelo.add_pipe(categorias)

# Armazenar os resultados
historico = []

In [13]:
from spacy.training import Example

# Iniciar o treinamento
modelo.begin_training()

for epoca in range(3):
    random.shuffle(db_final)  # Embaralha os dados de treinamento

    # Controlar o erro
    losses = {}

    # Processar os lotes
    for batch in spacy.util.minibatch(db_final, size=512):
        textos = [modelo.make_doc(texto) for texto, _ in batch]
        annotations = [{"cats": entities} for _, entities in batch]
        
        # Criar os exemplos no formato novo
        exemplos = [Example.from_dict(doc, annotation) for doc, annotation in zip(textos, annotations)]
        
        # Atualizar o modelo com os exemplos
        modelo.update(examples=exemplos, losses=losses)
        historico.append(losses)